### Juntando antiga(5%) com a nova 

In [1]:
import random
import numpy as np
import pandas as pd

def gerar_cromossomo():
    cromossomo = []
    
    # Gera posições iniciais (0-29)
    posicoes_iniciais = list(range(30))
    random.shuffle(posicoes_iniciais)
    cromossomo.extend(posicoes_iniciais)
    
    # Gera posições finais (0-29)
    posicoes_finais = list(range(30))
    random.shuffle(posicoes_finais)
    cromossomo.extend(posicoes_finais)
    
    # Gera ordem de movimentação (0-29)
    ordem_movimentacao = list(range(30))
    random.shuffle(ordem_movimentacao)
    cromossomo.extend(ordem_movimentacao)
    
    return cromossomo

def cromossomo_para_dataframe(cromossomo):
    container = list(range(30))
    posicao_inicial = cromossomo[:30]
    posicao_final = cromossomo[30:60]
    ordem_movimentacao = cromossomo[60:90]
    
    df = pd.DataFrame({
        'Container': container,
        'Posição Inicial': posicao_inicial,
        'Posição Final': posicao_final,
        'Ordem de Movimentação': ordem_movimentacao
    })
    
    # Ordenar o DataFrame pela coluna 'Ordem de Movimentação'
    df = df.sort_values(by='Ordem de Movimentação')
    
    return df

def inicializar_populacao(tamanho_populacao):
    populacao = []
    for _ in range(tamanho_populacao):
        cromossomo = gerar_cromossomo()
        populacao.append(cromossomo)
    return populacao

def verifica_repetidos(cromossomo):
    intervalos = [
        (0, 29),
        (30, 59),
        (60, 89)
    ]
    
    for intervalo in intervalos:
        valores = cromossomo[intervalo[0]:intervalo[1]+1]
        if len(valores) != len(set(valores)):
            return True
    
    return False

def preencher_porto(df_cromossomo):
    # Inicializar o porto e balsa
    porto = np.zeros((2, 5, 3), dtype=int)
    balsa = np.zeros((3, 4, 3), dtype=int)
    porto_preenchido = np.ones((2, 5, 3), dtype=int)
    cal_movimentos = 0
    
    # Preencher o porto com os dados do cromossomo
    ordem_inicial = []
    ordem_final = []

    df_cromossomo = df_cromossomo.sort_values(by='Posição Inicial')
    for _, linha in df_cromossomo.iterrows():
        ordem_inicial.append(linha['Container'])
    idx = 0
    for andar in range(2):
        for x in range(5):
            for y in range(3):
                if idx < 30:
                    porto[andar, x, y] = ordem_inicial[idx]
                    idx += 1
    
    df_cromossomo = df_cromossomo.sort_values(by='Ordem de Movimentação')
    # Realizar movimentações
    for _, linha in df_cromossomo.iterrows():
        container = linha['Container']
        a_pos_inicial = linha['Posição Inicial']
        a_pos_final = linha['Posição Final']
        pos_inicial = linha['Posição Inicial'] - 1
        pos_final = linha['Posição Final'] - 1
        
        andar_ini, x_ini, y_ini = pos_inicial // 15, (pos_inicial % 15) // 3, (pos_inicial % 15) % 3
        andar_fin, x_fin, y_fin = pos_final // 12, (pos_final % 12) // 3, (pos_final % 12) % 3
        
        # Verificar se podemos remover do porto preenchido
        pode_mover = True
        if andar_ini < 1:
            if porto_preenchido[andar_ini + 1, x_ini, y_ini] == 1:
                pode_mover = False
        
        # Verificar se podemos adicionar à balsa
        if andar_fin > 0:
            if balsa[andar_fin - 1, x_fin, y_fin] == 0:
                pode_mover = False

        # Verifica se o lugar está ocupado
        if balsa[andar_fin , x_fin, y_fin] == 1:
            pode_mover = False

                
        if pode_mover:
            # Atualizar porto_preenchido e balsa
            porto_preenchido[andar_ini, x_ini, y_ini] = 0
            balsa[andar_fin, x_fin, y_fin] = 1
            cal_movimentos += (
                              ((3-andar_ini)+(3-andar_fin)) #alturas
                              + (abs(x_ini - x_fin))
                              + ((2-y_ini)+(y_fin))
                              )
        else:
            cal_movimentos += 1000
            

    return cal_movimentos

def torneio(populacao, fitness, k=3):
    selecionados = random.sample(list(zip(populacao, fitness)), k)
    selecionados.sort(key=lambda x: x[1])
    return selecionados[0][0]

def cruzamento_pmx(pai1, pai2, inicio, fim):
    tamanho = fim - inicio + 1
    filho1 = pai1[:inicio] + [-1] * tamanho + pai1[fim+1:]
    filho2 = pai2[:inicio] + [-1] * tamanho + pai2[fim+1:]
    
    mapeamento1 = pai1[inicio:fim+1]
    mapeamento2 = pai2[inicio:fim+1]
    
    # Copiar o segmento
    filho1[inicio:fim+1] = mapeamento2
    filho2[inicio:fim+1] = mapeamento1
    
    # Resolver conflitos para filho1
    for i in range(inicio):
        if filho1[i] in mapeamento2:
            idx = mapeamento2.index(filho1[i])
            while mapeamento1[idx] in mapeamento2:
                idx = mapeamento2.index(mapeamento1[idx])
            filho1[i] = mapeamento1[idx]
    
    for i in range(fim+1, len(pai1)):
        if filho1[i] in mapeamento2:
            idx = mapeamento2.index(filho1[i])
            while mapeamento1[idx] in mapeamento2:
                idx = mapeamento2.index(mapeamento1[idx])
            filho1[i] = mapeamento1[idx]
    
    # Resolver conflitos para filho2
    for i in range(inicio):
        if filho2[i] in mapeamento1:
            idx = mapeamento1.index(filho2[i])
            while mapeamento2[idx] in mapeamento1:
                idx = mapeamento1.index(mapeamento2[idx])
            filho2[i] = mapeamento2[idx]
    
    for i in range(fim+1, len(pai2)):
        if filho2[i] in mapeamento1:
            idx = mapeamento1.index(filho2[i])
            while mapeamento2[idx] in mapeamento1:
                idx = mapeamento1.index(mapeamento2[idx])
            filho2[i] = mapeamento2[idx]
    
    return filho1, filho2

def cruzamento(pai, mae):
    filho1, filho2 = pai[:], mae[:]
    
    # Cruzamento para as posições iniciais (0-29)
    ponto_corte1 = random.randint(0, 29)
    ponto_corte2 = random.randint(0, 29)
    inicio, fim = min(ponto_corte1, ponto_corte2), max(ponto_corte1, ponto_corte2)
    filho1[:30], filho2[:30] = cruzamento_pmx(pai[:30], mae[:30], inicio, fim)
    
    # Cruzamento para as posições finais (30-59)
    ponto_corte1 = random.randint(30, 59)
    ponto_corte2 = random.randint(30, 59)
    inicio, fim = min(ponto_corte1, ponto_corte2) - 30, max(ponto_corte1, ponto_corte2) - 30
    filho1[30:60], filho2[30:60] = cruzamento_pmx(pai[30:60], mae[30:60], inicio, fim)
    
    # Cruzamento para a ordem de movimentação (60-89)
    ponto_corte1 = random.randint(60, 89)
    ponto_corte2 = random.randint(60, 89)
    inicio, fim = min(ponto_corte1, ponto_corte2) - 60, max(ponto_corte1, ponto_corte2) - 60
    filho1[60:90], filho2[60:90] = cruzamento_pmx(pai[60:90], mae[60:90], inicio, fim)
    
    return filho1, filho2

def mutacao(cromossomo, taxa_mutacao=0.01):
    if random.random() < taxa_mutacao:
        part = random.randint(0, 2)
        idx1, idx2 = random.sample(range(30), 2)
        if part == 0:
            cromossomo[idx1], cromossomo[idx2] = cromossomo[idx2], cromossomo[idx1]
        elif part == 1:
            cromossomo[30+idx1], cromossomo[30+idx2] = cromossomo[30+idx2], cromossomo[30+idx1]
        else:
            cromossomo[60+idx1], cromossomo[60+idx2] = cromossomo[60+idx2], cromossomo[60+idx1]
    return cromossomo

def algoritmo_genetico(tamanho_populacao, geracoes, taxa_mutacao):
    populacao = inicializar_populacao(tamanho_populacao)
    fitness = [preencher_porto(cromossomo_para_dataframe(cromossomo)) for cromossomo in populacao]
    
    for geracao in range(geracoes):
        nova_populacao = []
        while len(nova_populacao) < tamanho_populacao:
            pai = torneio(populacao, fitness)
            mae = torneio(populacao, fitness)
            
            filho1, filho2 = cruzamento(pai, mae)
            filho1 = mutacao(filho1, taxa_mutacao)
            filho2 = mutacao(filho2, taxa_mutacao)
            
            nova_populacao.append(filho1)
            nova_populacao.append(filho2)
        
        # Combinar 5% melhor da geração antiga com a metade melhor da nova geração
        populacao.extend(nova_populacao)
        fitness = [preencher_porto(cromossomo_para_dataframe(cromossomo)) for cromossomo in populacao]
        
        # Ordenar pela fitness e selecionar os melhores
        populacao_ordenada = sorted(zip(populacao, fitness), key=lambda x: x[1])
        
        melhor_5_porcento = int(tamanho_populacao * 0.05)
        nova_metade = tamanho_populacao - melhor_5_porcento
        
        populacao = [cromossomo for cromossomo, _ in populacao_ordenada[:melhor_5_porcento]] + [cromossomo for cromossomo, _ in populacao_ordenada[melhor_5_porcento:melhor_5_porcento+nova_metade]]
        fitness = [fit for _, fit in populacao_ordenada[:melhor_5_porcento]] + [fit for _, fit in populacao_ordenada[melhor_5_porcento:melhor_5_porcento+nova_metade]]
        
        melhor_fit = min(fitness)
        print(f"Geração {geracao+1}: Melhor fitness = {melhor_fit}")
    
    melhor_cromossomo = populacao[fitness.index(min(fitness))]
    return melhor_cromossomo, min(fitness)

# Configurações
tamanho_populacao = 100
geracoes = 200
taxa_mutacao = 0.1

melhor_cromossomo, melhor_fit = algoritmo_genetico(tamanho_populacao, geracoes, taxa_mutacao)
print(f"Melhor cromossomo encontrado: {melhor_cromossomo}")
print(f"Melhor fitness: {melhor_fit}")


Geração 1: Melhor fitness = 12153
Geração 2: Melhor fitness = 11168
Geração 3: Melhor fitness = 11168
Geração 4: Melhor fitness = 10164
Geração 5: Melhor fitness = 10164
Geração 6: Melhor fitness = 10164
Geração 7: Melhor fitness = 9185
Geração 8: Melhor fitness = 9179
Geração 9: Melhor fitness = 9176
Geração 10: Melhor fitness = 8187
Geração 11: Melhor fitness = 7197
Geração 12: Melhor fitness = 6207
Geração 13: Melhor fitness = 5215
Geração 14: Melhor fitness = 5215
Geração 15: Melhor fitness = 4224
Geração 16: Melhor fitness = 4224
Geração 17: Melhor fitness = 4220
Geração 18: Melhor fitness = 4220
Geração 19: Melhor fitness = 4216
Geração 20: Melhor fitness = 4216
Geração 21: Melhor fitness = 4216
Geração 22: Melhor fitness = 4213
Geração 23: Melhor fitness = 4213
Geração 24: Melhor fitness = 4213
Geração 25: Melhor fitness = 4211
Geração 26: Melhor fitness = 4210
Geração 27: Melhor fitness = 4208
Geração 28: Melhor fitness = 4208
Geração 29: Melhor fitness = 4207
Geração 30: Melho

#### Verifica se ha numeros repetidos


In [3]:
def verifica_repetidos(cromossomo):
    intervalos = [
        (0, 29),
        (30, 59),
        (60, 89)
    ]
    
    for intervalo in intervalos:
        valores = cromossomo[intervalo[0]:intervalo[1]+1]
        if len(valores) != len(set(valores)):
            return True
    
    return False

if verifica_repetidos(melhor_cromossomo):
    print("Valores repetidos encontrados!")
else:
    print("Não há valores repetidos.")

df_cromossomo = cromossomo_para_dataframe(melhor_cromossomo)
print("DataFrame do Cromossomo:")
print(df_cromossomo)


Não há valores repetidos.
DataFrame do Cromossomo:
    Container  Posição Inicial  Posição Final  Ordem de Movimentação
25         25               21              1                      0
24         24               22              3                      1
13         13               25             11                      2
12         12               17             13                      3
3           3                7              9                      4
9           9               26              4                      5
23         23               24              8                      6
6           6               10             16                      7
17         17               15             26                      8
7           7               19              2                      9
21         21                6              6                     10
1           1               16              5                     11
0           0               18             17       

### Simular Melhor cromomossomo

In [4]:
def Simular_preencher_porto(df_cromossomo):
    # Inicializar o porto e balsa
    porto = np.zeros((2, 5, 3), dtype=int)
    balsa = np.zeros((3, 4, 3), dtype=int)
    porto_preenchido = np.ones((2, 5, 3), dtype=int)
    
    
    # Preencher o porto com os dados do cromossomo
    ordem_inicial = []
    ordem_final = []
    

    df_cromossomo = df_cromossomo.sort_values(by='Posição Inicial')
    for _, linha in df_cromossomo.iterrows():
        ordem_inicial.append(linha['Container'])
    idx = 0
    for andar in range(2):
        for x in range(5):
            for y in range(3):
                if idx < 30:
                    porto[andar, x, y] = ordem_inicial[idx]
                    idx += 1
    
    df_cromossomo = df_cromossomo.sort_values(by='Ordem de Movimentação')
    # Realizar movimentações
    for _, linha in df_cromossomo.iterrows():
        container = linha['Container']
        a_pos_inicial = linha['Posição Inicial']
        a_pos_final = linha['Posição Final']
        pos_inicial = linha['Posição Inicial'] 
        pos_final = linha['Posição Final'] 
        
        andar_ini, x_ini, y_ini = pos_inicial // 15, (pos_inicial % 15) // 3, (pos_inicial % 15) % 3
        andar_fin, x_fin, y_fin = pos_final // 12, (pos_final % 12) // 3, (pos_final % 12) % 3
        
        # Verificar se podemos remover do porto preenchido
        pode_mover = True
        if andar_ini < 1:
            if porto_preenchido[andar_ini + 1, x_ini, y_ini] == 1:
                pode_mover = False
        
        # Verificar se podemos adicionar à balsa
        if andar_fin > 0:
            if balsa[andar_fin - 1, x_fin, y_fin] == 0:
                pode_mover = False
        
        if pode_mover:
            # Atualizar porto_preenchido e balsa
            porto_preenchido[andar_ini, x_ini, y_ini] = 0
            balsa[andar_fin, x_fin, y_fin] = 1
            
            print(f"Movendo container {container} de ({a_pos_inicial})({pos_inicial})({andar_ini}, {x_ini}, {y_ini}) para ({a_pos_final})({pos_final})({andar_fin}, {x_fin}, {y_fin})")
            print("Estado atual do Porto Preenchido:")
            print(porto_preenchido)
            print("Estado atual da Balsa:")
            print(balsa)
            print("\n")
        else:
            print(f"não moveu container {container} de ({a_pos_inicial})({pos_inicial})({andar_ini}, {x_ini}, {y_ini}) para ({a_pos_final})({pos_final})({andar_fin}, {x_fin}, {y_fin})")


Simular_preencher_porto(cromossomo_para_dataframe(melhor_cromossomo))

Movendo container 25 de (21)(21)(1, 2, 0) para (1)(1)(0, 0, 1)
Estado atual do Porto Preenchido:
[[[1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]]

 [[1 1 1]
  [1 1 1]
  [0 1 1]
  [1 1 1]
  [1 1 1]]]
Estado atual da Balsa:
[[[0 1 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]]


Movendo container 24 de (22)(22)(1, 2, 1) para (3)(3)(0, 1, 0)
Estado atual do Porto Preenchido:
[[[1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]]

 [[1 1 1]
  [1 1 1]
  [0 0 1]
  [1 1 1]
  [1 1 1]]]
Estado atual da Balsa:
[[[0 1 0]
  [1 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]]


Movendo container 13 de (25)(25)(1, 3, 1) para (11)(11)(0, 3, 2)
Estado atual do Porto Preenchido:
[[[1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]]

 [[1 1 1]
  [1 1 1]
  [0 0 1]
  [1 0 1]
  [1 1 1]]]
Estado atual da Balsa:
[[[0 1 0]
  [1 0 0]
  [0 0 0]
  [0 0 1]]

 [[0 0 0]
  [0 0 0]
  [0 0 0

In [ ]:
pos = 27
andar_ini, x_ini, y_ini = pos // 12, (pos % 12) // 3, (pos % 12) % 3
print(andar_ini," ",x_ini," ",y_ini)

2   1   0
